In [1]:
import os

In [2]:
pwd

'd:\\PW_DS\\Deep Learning\\End-To-End-Plant-Disease-Classification-using-CNN\\research'

In [3]:
os.chdir('..')

In [4]:
pwd

'd:\\PW_DS\\Deep Learning\\End-To-End-Plant-Disease-Classification-using-CNN'

## Entity

In [5]:
from dataclasses import  dataclass
from pathlib import  Path

In [6]:
@dataclass(frozen=True)
class Training:
    root_dir: Path
    model_path: Path
    model_metrics_path: Path
    data: Path
    updated_model_path: Path
    INPUT_SHAPE: list
    BATCH_SIZE: int
    SHUFFLE: bool
    VALIDATION_SPLIT: float
    LABEL_MODEL: str
    EPOCHS: int
    AUGMENTED: bool

    

## Config Manager

In [7]:
from plant_disease_clf.utils.common import  create_directories, read_yaml
from plant_disease_clf.constants import  *

In [8]:
class ConfigManager:
    def __init__(self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_training_config(self):

        self.config = self.config.model_training
        self.params = self.params.resnet50

        create_directories([self.config.root_dir])

        return Training (
            root_dir = Path(self.config.root_dir),
            model_path = Path(self.config.model_path),
            model_metrics_path = Path(self.config.model_metrics_path),
            data = Path(self.config.data),
            updated_model_path = Path(self.config.updated_model_path),
            INPUT_SHAPE = self.params.INPUT_SHAPE,
            BATCH_SIZE = self.params.BATCH_SIZE,
            SHUFFLE = self.params.SHUFFLE,
            VALIDATION_SPLIT = self.params.VALIDATION_SPLIT,
            LABEL_MODEL = self.params.LABEL_MODEL,
            EPOCHS = self.params.EPOCHS,
            AUGMENTED = self.params.AUGMENTED
        )



## Components

In [9]:
import os,sys
from pathlib import Path
from plant_disease_clf.logger import  logging
from plant_disease_clf.exception import  CustomException

from plant_disease_clf.utils.common import  save_json

import tensorflow as tf
from plant_disease_clf.pipeline.stage_03_prepare_callbacks import  CallbacksPipeline

In [10]:
# class ModelTraining:
#     def __init__(self, config: Training):
#         self.config = config
    
#     def get_updated_model(self):
#         return tf.keras.models.load_model(self.config.updated_model_path)


#     def train(self):
#         try:
#             logging.info(f"Training the model with the following config: {self.config}")

#             callbacks = CallbacksPipeline()
#             callbacks_list = callbacks.main()

# # -------------------------------------------------------------------------------------------------------------------------------------
            # train_data, val_data = tf.keras.utils.image_dataset_from_directory(
            #     directory = self.config.data,
            #     labels = "inferred",
            #     label_mode = self.config.LABEL_MODEL,
            #     batch_size = self.config.BATCH_SIZE,
            #     image_size = self.config.INPUT_SHAPE,
            #     shuffle = self.config.SHUFFLE,
            #     seed = 42,  #--------------------- add this for splitting the data
            #     validation_split = self.config.VALIDATION_SPLIT,
            #     subset = self.config.SUBSET,
            #     interpolation = "bilinear",
            # )
            # train_data_samples = tf.data.experimental.cardinality(train_data).numpy()
            # val_data_samples = tf.data.experimental.cardinality(val_data).numpy()
# # -------------------------------------------------------------------------------------------------------------------------------------
#             data_generator_kwargs = dict(
#                 rescale = 1./255,
#                 validation_split = 0.20
#             )

#             data_flow_kwargs = dict(
#                 target_size = self.config.INPUT_SHAPE[:-1],
#                 batch_size = self.config.BATCH_SIZE,
#                 interpolation = 'bilinear'
#             )


#             valid_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
#                 **data_generator_kwargs
#             )

#             val_data = valid_data_gen.flow_from_directory(
#                 directory = self.config.data,
#                 class_mode = self.config.LABEL_MODEL,
#                 shuffle = self.config.SHUFFLE,
#                 seed = 42,  #--------------------- add this for splitting the data
#                 subset = 'validation',
#                 # interpolation = "bilinear",
#                 **data_flow_kwargs
#             )

#             if self.config.AUGMENTED:
#                 train_data_gen= tf.keras.preprocessing.image.ImageDataGenerator(
#                     rotation_range = 40,
#                     horizontal_flip = True,
#                     width_shift_range = 0.2,
#                     height_shift_range = 0.2,
#                     shear_range = 0.2,
#                     zoom_range = 0.2,
#                     **data_generator_kwargs
#                 )

#             else:
#                 train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
#                     **data_generator_kwargs
#                 )

#             train_data = train_data_gen.flow_from_directory(
#                 directory = self.config.data,
#                 class_mode = self.config.LABEL_MODEL,
#                 shuffle = True, ### 
#                 # seed = 42,  #--------------------- add this for splitting the data
#                 subset = 'training',
#                 # interpolation = "bilinear",
#                 **data_flow_kwargs
#             )

#             model = self.get_updated_model()

#             print(callbacks_list)
#             print(model.summary())

#             # history = model.fit(
#             #     train_data,
#             #     epochs = self.config.EPOCHS,
#             #     validation_data = val_data,
#             #     callbacks = callbacks_list
#             # )

            
#             self.steps_per_epoch = train_data.samples // self.config.BATCH_SIZE

#             self.validation_steps = val_data.samples // self.config.BATCH_SIZE 


#             history = model.fit(
#                 train_data,
#                 steps_per_epoch = self.steps_per_epoch,
#                 epochs = self.config.EPOCHS,
#                 validation_data = val_data,
#                 validation_steps = self.validation_steps,
#                 callbacks = callbacks_list
#             )

#             # tf.keras.models.save_model(model, self.config.model_path)
#             # save_json(self.config.model_metrics_path, history.history)
#             logging.info("Model training completed successfully")


#         except Exception as e:
#             logging.error(f"Model training failed: {e}")
#             raise CustomException(e,sys)




## ------------------------------------

In [26]:
class ModelTraining:
    def __init__(
        self,
        config: Training):

        self.config = config

    
    def get_updated_base_model(self):
        self.model =  tf.keras.models.load_model(self.config.updated_model_path)


    def train_valid_generator(self):

        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = self.config.VALIDATION_SPLIT
        )

        # test_data_generator_kwargs = dict(
        #     rescale = 1./255,
        #     validation_split = 0.20
        # )



        data_flow_kwargs = dict(
            target_size = self.config.INPUT_SHAPE[:-1],
            batch_size = self.config.BATCH_SIZE,
            # shuffle = True,
            interpolation = 'bilinear'
        )

        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )


        self.valid_generator = valid_data_generator.flow_from_directory(
            directory = self.config.data,
            subset = 'validation',
            shuffle = False,
            class_mode= self.config.LABEL_MODEL,
            **data_flow_kwargs
        )


        # test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        #     **test_data_generator_kwargs
        # )


        # self.test_generator = test_data_generator.flow_from_directory(
        #     data = self.valid_generator,
        #     subset = 'validation',
        #     shuffle = False,
        #     class_mode='categorical',
        #     **data_flow_kwargs
        # )



        if self.config.AUGMENTED:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **data_generator_kwargs
            )

        else:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                **data_generator_kwargs
            )
        
        self.train_generator = train_data_generator.flow_from_directory(
            directory = self.config.data,
            subset = 'training',
            shuffle = True,
            class_mode= self.config.LABEL_MODEL,
            **data_flow_kwargs
        )

    
    def get_data(self):
            self.train_data, self.val_data = tf.keras.utils.image_dataset_from_directory(
            directory = self.config.data,
            labels = "inferred",
            label_mode = self.config.LABEL_MODEL,
            batch_size = self.config.BATCH_SIZE,
            image_size = self.config.INPUT_SHAPE[:-1],
            shuffle = self.config.SHUFFLE,
            seed = 42,  #--------------------- add this for splitting the data
            validation_split = self.config.VALIDATION_SPLIT,
            subset = 'both',
            interpolation = "bilinear",

            )
            self.train_data_samples = tf.data.experimental.cardinality(self.train_data).numpy()
            self.val_data_samples = tf.data.experimental.cardinality(self.val_data).numpy()

            # print(self.train_data['labels'][0], self.val_data['labels'][0])
            print(self.train_data.class_names)


    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)
    

    def train(self):
        self.steps_per_epoch = self.train_data_samples// self.config.BATCH_SIZE

        self.validation_steps = self.val_data_samples // self.config.BATCH_SIZE

        callbacks = CallbacksPipeline()
        callbacks_list = callbacks.main()

        history = self.model.fit(
            self.train_data,
            steps_per_epoch = self.steps_per_epoch,
            epochs = self.config.EPOCHS,
            validation_data = self.val_data,
            validation_steps = self.validation_steps,
            callbacks = callbacks_list
        )

        save_json(self.config.model_metrics_path, history.history)
        

        self.save_model(
            path = self.config.model_path,
            model =self.model
        )




## Pipeline

In [27]:


try:
    config_manager = ConfigManager()
    config = config_manager.get_model_training_config()
    training = ModelTraining(config)
    training.get_updated_base_model()
    # training.train_valid_generator()
    training.get_data()
    training.train()

except Exception as e:
    raise CustomException(e,sys)

Found 35614 files belonging to 20 classes.
Using 28492 files for training.
Using 7122 files for validation.
['01_Apple_Apple scab', '02_Apple_Black_rot', '03_Apple_Cedar apple rust', '04_Apple_healthy', '05_Corn_(maize) healthy', '06_Corn_Blight in corn Leaf', '07_Corn_Gray Leaf Spot in corn Leaf', '08_Grape_Black rot', '09_Grape_Esca Black Measles', '10_Grape_healthy', '11_Grape_Leaf blight Isariopsis Leaf Spot', '12_Potato_Early blight', '13_Potato_healthy', '14_Potato_Late blight', '15_Tomato_Bacterial spot', '16_Tomato_Early blight', '17_Tomato_healthy', '18_Tomato_Late blight', '19_Tomato_Septoria leaf spot', '20_Tomato_Tomato mosaic virus']
Epoch 1/2
27/27 [==============================] - ETA: 0s - loss: 0.2620 - accuracy: 0.9630

CustomException: Error occured in python script name [C:\Users\Maaz\AppData\Local\Temp\ipykernel_27924\3353754261.py] line number [9] error occured is [{{function_node __wrapped__OneHot_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[25690112,30] and type double on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:OneHot] name: ].